# This notebook has been productionized in .\tensor-hero\Preprocessing\chart_functions.py

see function chart2onehot()

In [1]:
import sys
import numpy as np
from pathlib import Path
from itertools import combinations
# Define path to chart
d = str(Path().resolve().parent)
sys.path.insert(1, d + r'\Preprocessing')

from chart_functions import chart2dict, get_configuration

# Import the processed .chart file
chartpath = d+'\Preprocessing\Chart Files\degausser_notes.chart'
folderpath = d+'\Prototypes\Toy Training Data\Brand New - Degausser (Chezy)'

notes, song_metadata, time_signatures, BPMs = chart2dict(chartpath)
config = get_configuration(folderpath)

In [2]:
# Create a dictionary where the keys are the tick values and the values
    # are a list of notes corresponding to the data
coded_notes_0 = {}

# Loop through song one note at a time, processing along the way
for i in range(len(notes['tick'])):
    #if notes['N_S'][i] == 'S':  # if the token is a star power indicator, skip it
    if notes['N_S'] == 'S':  # if the token is a star power indicator, skip it
        continue

    if notes['tick'][i] not in coded_notes_0:  # If the key is not in the dictionary
        coded_notes_0[notes['tick'][i]] = []                   # Create empty list

    if notes['duration'][i] == 0:  # If the note is not held
        coded_notes_0[notes['tick'][i]].append(int(notes['note'][i]))  # Add note to list
    else:  # If the note is held
        if (notes['tick'][i] + notes['duration'][i]) not in coded_notes_0:  # If the key is not in the dictionary
            coded_notes_0[notes['tick'][i] + notes['duration'][i]] = []     # Create empty list
        # Add the note with a hold key
        # Hold key is the code of the note + 10 (so 11 indicates release green, 12 is release red, etc.)
        coded_notes_0[notes['tick'][i]].append(int(notes['note'][i]) + 10)  # Add note to list

        # Add a release key at time step when note will be released.
        # Release key is the code of the note + 20 (so 21 indicates release green, 22 is release red, etc.)
        coded_notes_0[notes['tick'][i] + notes['duration'][i]].append(int(notes['note'][i]) + 20)  # Add note to list

In [3]:
print(coded_notes_0)

{605: [1], 681: [1], 792: [1], 903: [1], 1013: [1, 2], 1124: [1], 1234: [1], 1345: [1, 2], 1455: [1], 1566: [1], 1677: [1], 1787: [1], 1898: [1, 2], 2008: [1], 2119: [1, 2], 2230: [7], 2340: [0, 2], 2451: [0], 2561: [0], 2672: [0], 2783: [0], 2893: [0], 3004: [0], 3114: [0], 3225: [0], 3336: [0, 2], 3446: [0, 2], 3557: [0, 2], 3667: [0, 2], 3778: [0, 2], 3888: [0, 2], 3999: [7], 4110: [1], 4220: [1], 4331: [1], 4441: [1], 4552: [1, 2], 4663: [1], 4773: [1], 4884: [1, 2, 4], 4994: [1], 5105: [1], 5216: [1], 5326: [1], 5437: [1, 2], 5547: [1], 5658: [1, 2, 4], 5769: [7], 5879: [0, 2], 5990: [0], 6100: [0], 6211: [0], 6322: [0], 6432: [0], 6543: [0], 6653: [0, 2], 6764: [0], 6874: [0, 2], 6985: [0, 2], 7096: [0], 7206: [0, 2], 7317: [0, 2], 7427: [0, 2], 7538: [7], 7649: [11, 12], 7815: [21], 9418: [22, 10], 7870: [2], 7980: [0], 8091: [1], 8202: [4], 8312: [1], 8423: [1], 8533: [3], 8644: [1], 8755: [1], 8865: [12], 9363: [22], 9584: [20], 9639: [1], 9750: [7], 9860: [0], 9971: [2], 1008

## how to read coded_notes
coded_notes is a dictionary with the keys representing ticks and values decoded as follows:

| Value | Interpretation | Value | Interpretation |
| --- | --- | ---|---|
| 0 | green note | 5 | force note flag
| 10 | green note hold | 6 | tap note flag
| 20 | green note release |
| 1 | red note | 
| 11 | red note hold |
| 21 | red note release |
| 2 | yellow note |
| 12 | yellow note hold |
| 22 | yellow note release | 
| 3 | blue note | 
| 13 | blue note hold |
| 23 | blue note release |
| 4 | orange note |
| 14 | orange note hold |
| 24 | orange note release |
| 7 | open note

The flags are always listed after a note value. For example, a 10 followed by a 5 says 'hold a tapped green note.'

## next steps
One hot encode the 'bag of notes'. Translate coded_notes into a tensor with the features *timestamp, note event*, with a timestamp existing for every 10ms of the song.

We will translate as follows:

| Value | Interpretation | regular/force/tap |
| --- | --- | ---|
|0 | no note |
| 10 | green note | regular |
| 11 | green note | force |
| 12 | green note | tap |
| 13 | green note hold| regular |
| 14| green note hold| force |
| 15 | green note hold| tap |
| 16 | green note release |
| 17 | red note | regular |
| 18 | red note | force |
| 19 | red note | tap |
| 20 | red note hold | regular |
| 21 | red note hold | force |
| 22 | red note hold | tap |
| 23 | red note release |
| 24 | yellow note | regular |
| 25 | yellow note | force |
| 26 | yellow note | tap |
| 27 | yellow note hold | regular |
| 28 | yellow note hold | force |
| 29 | yellow note hold | tap |
| 30 | yellow note release | 
| 31 | blue note |regular |
| 32 | blue note | force|
| 33 | blue note | tap|
| 34 | blue note hold | regular |
| 35 | blue note hold |force
| 36 | blue note hold |tap
| 37 | blue note release |
| 38 | orange note |regular
| 39 | orange note |force
| 40 | orange note |tap
| 41 | orange note hold|regular
| 42 | orange note hold|force
| 43 | orange note hold|tap
| 44 | orange note release |
| 45 | open note | regular
| 46 | open note | force
| 47 | open note | tap

It is possible to have multiple notes per timestep (i.e. a chord), these will be dealt with in a subsequent step. First, this translation will be dubbed coded_notes_1

In [4]:
# Define function to map notes to their proper representation
def map_notes_0(note_array, note_type):
    '''
    map_notes_0 maps a note array from the initial representation to an intermediate
    representation that can be processed later into a full one hot representation.
    The note_array should be preprocessed so that force and tap flags are removed
    - note_array = array of notes
    - type = 'regular', 'force', or 'tap'
    '''
    assert note_type in ['regular', 'force', 'tap'], 'note_type should be "regular", "force", or "tap"'
    
    for i in range(len(note_array)):
        if note_array[i] == 0:
            note_array[i] = 10
        elif note_array[i] == 10:
            note_array[i] = 13
        elif note_array[i] == 20:
            note_array[i] = 16
        elif note_array[i] == 1:
            note_array[i] = 17            
        elif note_array[i] == 11:
            note_array[i] = 20
        elif note_array[i] == 21:
            note_array[i] = 23
        elif note_array[i] == 2:
            note_array[i] = 24
        elif note_array[i] == 12:
            note_array[i] = 27
        elif note_array[i] == 22:
            note_array[i] = 30
        elif note_array[i] == 3:
            note_array[i] = 31
        elif note_array[i] == 13:
            note_array[i] = 34
        elif note_array[i] == 23:
            note_array[i] = 37
        elif note_array[i] == 4:
            note_array[i] = 38
        elif note_array[i] == 14:
            note_array[i] = 41
        elif note_array[i] == 24:
            note_array[i] = 44
        elif note_array[i] == 7:
            note_array[i] = 45
        else:
            raise NameError('Error: note encoded incorrectly')

        if note_type == 'regular':
            continue
        elif note_type == 'force':
            note_array[i] += 1
        elif note_type == 'tap':
            note_array[i] += 2

    return note_array

In [5]:
# coded_notes_1 will hold intermediate values of coded_notes
coded_notes_1 = {}

for x in coded_notes_0.keys():
    if 5 in coded_notes_0[x]:    # If a force note
        coded_notes_0[x].remove(5)
        coded_notes_1[x] = map_notes_0(coded_notes_0[x], 'force')
    elif 6 in coded_notes_0[x]:  # If a tap note
        coded_notes_0[x].remove(6)
        coded_notes_1[x] = map_notes_0(coded_notes_0[x], 'tap')
    else:                        # If a regular note
        coded_notes_1[x] = map_notes_0(coded_notes_0[x], 'regular')


###    FIXME

There is an error being thrown for the [30, 13] which means release yellow and hold green note at the same time
Need to find a workaround for this

    FIXED

Now the release note code is bumped to the next available tick.

In [6]:
# To one hot encode, we need to generate a list of all the possible unique values each note event could take on
g = list(range(10,17))
r = list(range(17,24))
y = list(range(24,31))
b = list(range(31,38))
o = list(range(38,45))

note_vals = [g, r, y, b, o]
note_vals = np.array(note_vals)

rr = note_vals[:,0]  # regular regular
rf = note_vals[:,1]  # regular forced
rt = note_vals[:,2]  # regular tapped
hr = note_vals[:,3]  # held regular
hf = note_vals[:,4]  # held forced
ht = note_vals[:,5]  # held tapped
release = note_vals[:,6]  # release
note_combos = [rr, rf, rt, hr, hf, ht, release]

all_combinations = []  # Will hold all possible note combinations
for combo_class in note_combos:
    for combo_length in range(1, len(combo_class)+1):
        for combo in list(combinations(combo_class, combo_length)):
            keystring = ''
            for element in combo:
                keystring += str(element)
            all_combinations.append(keystring)

# Add open notes
all_combinations.append('45')
all_combinations.append('46')
all_combinations.append('47')

In [7]:
# Sanity check, are all the values in all_combinations unique?
unique_vals = len(list(set(all_combinations)))
print(len(all_combinations) == unique_vals)
print(all_combinations)

True
['10', '17', '24', '31', '38', '1017', '1024', '1031', '1038', '1724', '1731', '1738', '2431', '2438', '3138', '101724', '101731', '101738', '102431', '102438', '103138', '172431', '172438', '173138', '243138', '10172431', '10172438', '10173138', '10243138', '17243138', '1017243138', '11', '18', '25', '32', '39', '1118', '1125', '1132', '1139', '1825', '1832', '1839', '2532', '2539', '3239', '111825', '111832', '111839', '112532', '112539', '113239', '182532', '182539', '183239', '253239', '11182532', '11182539', '11183239', '11253239', '18253239', '1118253239', '12', '19', '26', '33', '40', '1219', '1226', '1233', '1240', '1926', '1933', '1940', '2633', '2640', '3340', '121926', '121933', '121940', '122633', '122640', '123340', '192633', '192640', '193340', '263340', '12192633', '12192640', '12193340', '12263340', '19263340', '1219263340', '13', '20', '27', '34', '41', '1320', '1327', '1334', '1341', '2027', '2034', '2041', '2734', '2741', '3441', '132027', '132034', '132041', '1

In [8]:
# coded_notes_2 will map the coded_notes_1 values into the syntax of the values described by all_combinations

coded_notes_2 = {}

for x in coded_notes_1.keys():
    notestring = ''
    for note_event in coded_notes_1[x]:
        notestring += str(note_event)
    coded_notes_2[x] = notestring

In [9]:
# combo_dictionary will map the coded values to a simpler representation [1:len(all_combinations)]

combo_dictionary = {}
for i in range(1, len(all_combinations)+1):
    combo_dictionary[all_combinations[i-1]] = i

In [10]:
# coded_notes_3 translates the coded notes into the simpler representations given by combo_dictionary
# combo_dictionary codes the combinations from 1-220

def check_for_release_notes(x):
    '''
    Checks to see if there is a released note at tick x
    '''
    r_notes = ['16', '23', '30', '37', '44']
    r_in_x = []
    for r in (r_notes):
        if r in coded_notes_2[x]:
            r_in_x.append(r)

    if not r_in_x:
        return False
    else:
        return r_in_x

coded_notes_3 = {}
replaced = {'x' : [],  # If notes need to be shuffled for errors, this will be populated
            'replacement_digits' : [],
            'y' : [],
            'release_digits' : []}
for x in coded_notes_2.keys():
    try:
        coded_notes_3[x] = combo_dictionary[coded_notes_2[x]]  # If no error, insert combo into coded_notes_3
    except:
        if check_for_release_notes(x):  # If released note and new note coincide on a tick
            y = x+1
            while y in coded_notes_2:  # Choose an unoccupied tick in front of x
                y+=1
            
            # Parse the string and strip away release indicators
            replacement_digits = ''
            release_digits = ''
            code = ''
            for digit in coded_notes_2[x]:
                if not code:
                    code = digit
                    continue
                if len(code) < 2:
                    code += digit
                else:
                    code = digit
                    continue
                if code in ['16', '23', '30', '37', '44']:
                    release_digits += code
                else:
                    replacement_digits += code

            # Replace note
            replaced['x'].append(x)
            replaced['y'].append(y)
            replaced['release_digits'].append(release_digits)
            replaced['replacement_digits'].append(replacement_digits)            
            print('Release Notes Coincided at tick', x,': bumped to tick', y)
            

for i in range(len(replaced['x'])):
    coded_notes_2[replaced['x'][i]] = replaced['replacement_digits'][i]
    coded_notes_2[replaced['y'][i]] = replaced['release_digits'][i]
    try:
        coded_notes_3[replaced['x'][i]] = combo_dictionary[coded_notes_2[replaced['x'][i]]]
        coded_notes_3[replaced['y'][i]] = combo_dictionary[coded_notes_2[replaced['y'][i]]]
    except:
        raise NameError('Release notes are not in combination dictionary')

Release Notes Coincided at tick 9418 : bumped to tick 9419
Release Notes Coincided at tick 21804 : bumped to tick 21805
Release Notes Coincided at tick 46590 : bumped to tick 46591
Release Notes Coincided at tick 58976 : bumped to tick 58977
Release Notes Coincided at tick 73132 : bumped to tick 73133
Release Notes Coincided at tick 126996 : bumped to tick 126997
Release Notes Coincided at tick 145133 : bumped to tick 145134
Release Notes Coincided at tick 155738 : bumped to tick 155739


In [11]:
print(coded_notes_3)

{605: 2, 681: 2, 792: 2, 903: 2, 1013: 10, 1124: 2, 1234: 2, 1345: 10, 1455: 2, 1566: 2, 1677: 2, 1787: 2, 1898: 10, 2008: 2, 2119: 10, 2230: 218, 2340: 7, 2451: 1, 2561: 1, 2672: 1, 2783: 1, 2893: 1, 3004: 1, 3114: 1, 3225: 1, 3336: 7, 3446: 7, 3557: 7, 3667: 7, 3778: 7, 3888: 7, 3999: 218, 4110: 2, 4220: 2, 4331: 2, 4441: 2, 4552: 10, 4663: 2, 4773: 2, 4884: 23, 4994: 2, 5105: 2, 5216: 2, 5326: 2, 5437: 10, 5547: 2, 5658: 23, 5769: 218, 5879: 7, 5990: 1, 6100: 1, 6211: 1, 6322: 1, 6432: 1, 6543: 1, 6653: 7, 6764: 1, 6874: 7, 6985: 7, 7096: 1, 7206: 7, 7317: 7, 7427: 7, 7538: 218, 7649: 103, 7815: 188, 7870: 3, 7980: 1, 8091: 2, 8202: 5, 8312: 2, 8423: 2, 8533: 4, 8644: 2, 8755: 2, 8865: 96, 9363: 189, 9584: 187, 9639: 2, 9750: 218, 9860: 1, 9971: 3, 10082: 218, 10192: 1, 10303: 4, 10413: 1, 10524: 1, 10635: 98, 11133: 191, 11188: 95, 11354: 188, 11409: 3, 11519: 1, 11630: 2, 11741: 5, 11851: 2, 11962: 2, 12072: 4, 12183: 2, 12293: 2, 12404: 96, 12902: 189, 12957: 94, 13123: 187, 1317